In [1]:
%matplotlib inline

import geopandas as gpd
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import pandas as pd
import shapely
from geoalchemy2 import Geometry, WKTElement
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry.polygon import Polygon
from sqlalchemy import text

plt.style.use("ggplot")

# CL Vestland: Add catchment data to PostGIS

Add catchment data from the template here:

    critical_loads_2/cl_vestland/cl_vestland_stns.xlsx

In [2]:
# Connect to PostGIS
# If connecting from JupyterHub
eng = nivapy.da.connect_postgis(admin=True)

Username:  ········
Password:  ·······


Connection successful.


## 1. Add project

In [4]:
# Read projects
xl_path = r"../data/cl_vestland_stns.xlsx"
proj_df = pd.read_excel(xl_path, sheet_name="project")
assert len(proj_df) == 1

proj_df

,project_code,project_name,aquamonitor_id,contact,description
0,190246,CL Vestland,NaN,KAU,Critical loads calculations for Vestland


In [ ]:
## Add project
# proj_df.to_sql(
#    "projects", schema="niva", con=eng, if_exists="append", index=False,
# )

## 2. Add stations

In [5]:
# Read stations
xl_path = r"../data/cl_vestland_stns.xlsx"
stn_df = pd.read_excel(xl_path, sheet_name="stations")
stn_df["longitude"] = stn_df["longitude"].round(6)
stn_df["latitude"] = stn_df["latitude"].round(6)

stn_df.head()

,station_code,station_name,aquamonitor_id,longitude,latitude,fpath_or_id
0,Eks_Main,Eksingedal_Main,NaN,5.793674,60.728518,../shapefiles/Eksingedal_Main.shp
1,Eks_Opp,Eksingedal_Oppstr,NaN,5.905760,60.792344,../shapefiles/Eksingedal_Oppstr.shp
2,Eks_Side,Eksingedal_Side,NaN,5.806698,60.734305,../shapefiles/Eksingedal_Side.shp
3,Gud_Main,Guddal_Main,NaN,5.345209,61.311320,../shapefiles/Guddal_Main.shp
4,Gud_Opp,Guddal_Oppstr,NaN,5.590573,61.229553,../shapefiles/Guddal_Oppstr.shp


In [8]:
# Build geom
stn_gdf = gpd.GeoDataFrame(
    stn_df,
    crs="epsg:4326",
    geometry=gpd.points_from_xy(stn_df.longitude, stn_df.latitude),
).copy()
stn_gdf["geom"] = stn_gdf["geometry"].apply(lambda x: WKTElement(x.wkt, srid=4326))
del stn_gdf["fpath_or_id"], stn_gdf["geometry"]

stn_gdf.head()

,station_code,station_name,aquamonitor_id,longitude,latitude,geom
0,Eks_Main,Eksingedal_Main,NaN,5.793674,60.728518,POINT (5.793674 60.728518)
1,Eks_Opp,Eksingedal_Oppstr,NaN,5.905760,60.792344,POINT (5.90576 60.792344)
2,Eks_Side,Eksingedal_Side,NaN,5.806698,60.734305,POINT (5.806698 60.734305)
3,Gud_Main,Guddal_Main,NaN,5.345209,61.311320,POINT (5.345209 61.31132)
4,Gud_Opp,Guddal_Oppstr,NaN,5.590573,61.229553,POINT (5.590573 61.229553)


In [9]:
# # Add stations
# stn_gdf.to_sql(
#     "stations",
#     schema="niva",
#     con=eng,
#     if_exists="append",
#     index=False,
#     dtype={"geom": Geometry("POINT", srid=4326)},
#     method="multi",
#     chunksize=1000,
# )

## 3. Add project-stations

In [10]:
# Build table for projects-stations
# Get station IDs
stn_codes = tuple(stn_df["station_code"].unique())
sql = text("SELECT station_id FROM niva.stations " "WHERE station_code IN :stn_codes")
prst_df = pd.read_sql(sql, params={"stn_codes": stn_codes}, con=eng)

# Get project ID
proj_code = proj_df["project_code"].values[0]
sql = text("SELECT project_id FROM niva.projects " "WHERE project_code = :proj_code")
proj_id = pd.read_sql(sql, params={"proj_code": str(proj_code)}, con=eng)[
    "project_id"
].iloc[0]

prst_df["project_id"] = proj_id
prst_df.head()

,station_id,project_id
0,1260,2
1,1261,2
2,1262,2
3,1263,2
4,1264,2


In [11]:
# # Add projects-stations
# prst_df.to_sql(
#     "projects_stations",
#     schema="niva",
#     con=eng,
#     if_exists="append",
#     index=False,
#     method="multi",
#     chunksize=1000,
# )

## 4. Add catchments

In [13]:
# Loop over catchments
gdf_list = []
for idx, row in stn_df.iterrows():
    stn_code = row["station_code"]
    shp_path = row["fpath_or_id"]
    cat_gdf = gpd.read_file(shp_path)
    cat_gdf["dissolve"] = 1
    cat_gdf["station_code"] = stn_code
    cat_gdf = cat_gdf.dissolve(by="dissolve").reset_index()
    cat_gdf = cat_gdf[["station_code", "geometry"]]
    gdf_list.append(cat_gdf)

cat_gdf = pd.concat(gdf_list).reset_index(drop=True)
cat_gdf.head()

,station_code,geometry
0,Eks_Main,"POLYGON Z ((3082.020 6765260.340 0.000, 3036.7..."
1,Eks_Opp,"POLYGON Z ((7265.790 6767632.250 0.000, 7240.5..."
2,Eks_Side,"POLYGON Z ((6017.098 6773983.675 0.000, 5981.5..."
3,Gud_Main,"POLYGON Z ((-15898.619 6835765.865 0.000, -158..."
4,Gud_Opp,"POLYGON Z ((-1925.930 6823525.440 0.000, -1851..."


In [14]:
# Get station IDs from db
proj_id = 2
sql = text(
    "SELECT station_id, station_code FROM niva.stations "
    "WHERE station_id IN ( "
    "  SELECT station_id from niva.projects_stations "
    "  WHERE project_id = :proj_id)"
)
stn_ids = pd.read_sql(sql, params={"proj_id": proj_id}, con=eng)

# Join catchments
cat_gdf = cat_gdf.merge(stn_ids, on="station_code")

# Reproject to WGS84 GCS
cat_gdf = cat_gdf.to_crs("epsg:4326")

# Cast to multi
cat_gdf["geometry"] = [
    MultiPolygon([feature]) if type(feature) == Polygon else feature
    for feature in cat_gdf["geometry"]
]

# Convert 3D to 2D
cat_gdf["geom"] = cat_gdf["geometry"].apply(
    lambda x: shapely.wkb.loads(shapely.wkb.dumps(x, output_dimension=2))
)

# Tidy
cat_gdf["geom"] = cat_gdf["geom"].apply(lambda x: WKTElement(x.wkt, srid=4326))
cat_gdf = cat_gdf[["station_id", "geom"]]

cat_gdf.head()

,station_id,geom
0,1260,MULTIPOLYGON (((5.874700527116899 60.711371725...
1,1261,MULTIPOLYGON (((5.944435867691666 60.737576562...
2,1262,MULTIPOLYGON (((5.9057602930323 60.79234398739...
3,1263,MULTIPOLYGON (((5.342199208888013 61.311556719...
4,1264,MULTIPOLYGON (((5.632218260298241 61.221357793...


In [15]:
# # Add catchments
# cat_gdf.to_sql(
#     "catchments",
#     schema="niva",
#     con=eng,
#     if_exists="append",
#     index=False,
#     dtype={"geom": Geometry("MULTIPOLYGON", srid=4326)},
#     method="multi",
#     chunksize=1000,
# )